In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-pubmed-dataset-train-all-values/preprocessed_pubmed_train(1).xls


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRU
from sklearn.metrics import f1_score

# Load the dataset
pubmed_data = pd.read_csv('/kaggle/input/preprocessed-pubmed-dataset-train-all-values/preprocessed_pubmed_train(1).xls')

pubmed_data['article'] = pubmed_data['article'].fillna('').astype(str)
pubmed_data['abstract'] = pubmed_data['abstract'].fillna('').astype(str)

# Check for potential empty strings
pubmed_data['article'] = pubmed_data['article'].replace(r'^\s*$', '<EMPTY>', regex=True)
pubmed_data['abstract'] = pubmed_data['abstract'].replace(r'^\s*$', '<EMPTY>', regex=True)

# Hyperparameters
max_length = 512  # Based on your dataset's max token length
vocab_size = 20000  # Adjusted based on total unique tokens
embedding_dim = 128
hidden_units = 256
batch_size = 1
epochs = 3
max_iterations = 50

# Prepare the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(pubmed_data['article'])

# Convert text to sequences
article_sequences = tokenizer.texts_to_sequences(pubmed_data['article'])
abstract_sequences = tokenizer.texts_to_sequences(pubmed_data['abstract'])

# Padding
article_padded = pad_sequences(article_sequences, maxlen=max_length, padding='post', truncating='post')
abstract_padded = pad_sequences(abstract_sequences, maxlen=max_length, padding='post', truncating='post')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(article_padded, abstract_padded, test_size=0.2, random_state=42)

# Ensure y_train and y_test are reshaped correctly
y_train = np.expand_dims(y_train, -1)  # Shape: (batch_size, max_length, 1)
y_test = np.expand_dims(y_test, -1)    # Shape: (batch_size, max_length, 1)

# Model Definitions
def build_lstm_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        LSTM(hidden_units, return_sequences=True),  
        Dense(vocab_size, activation='softmax')  # Output shape should be (batch_size, max_length, vocab_size)
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_bilstm_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(LSTM(hidden_units, return_sequences=True)),  # Bi-directional LSTM
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_gru_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        GRU(hidden_units, return_sequences=True),  # Ensure GRU returns sequences
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Training the models
lstm_model = build_lstm_model()
bilstm_model = build_bilstm_model()
gru_model = build_gru_model()

lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
bilstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
88000/88000 ━━━━━━━━━━━━━━━━━━━━ 3201s 36ms/step - accuracy: 0.6618 - loss: 2.6658 - val_accuracy: 0.6642 - val_loss: 2.5222
Epoch 2/3
88000/88000 ━━━━━━━━━━━━━━━━━━━━ 3195s 36ms/step - accuracy: 0.6655 - loss: 2.4907 - val_accuracy: 0.6643 - val_loss: 2.4898
Epoch 3/3
88000/88000 ━━━━━━━━━━━━━━━━━━━━ 3196s 36ms/step - accuracy: 0.6648 - loss: 2.4554 - val_accuracy: 0.6639 - val_loss: 2.4876
Epoch 1/3
62325/88000 ━━━━━━━━━━━━━━━━━━━━ 20:39 48ms/step - accuracy: 0.6606 - loss: 2.7104

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



76131/88000 ━━━━━━━━━━━━━━━━━━━━ 9:33 48ms/step - accuracy: 0.6611 - loss: 2.6910

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

42765/88000 ━━━━━━━━━━━━━━━━━━━━ 23:24 31ms/step - accuracy: 0.6608 - loss: 2.7624

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



88000/88000 ━━━━━━━━━━━━━━━━━━━━ 3140s 36ms/step - accuracy: 0.6619 - loss: 2.7043 - val_accuracy: 0.6631 - val_loss: 2.5627
Epoch 2/3
88000/88000 ━━━━━━━━━━━━━━━━━━━━ 3137s 36ms/step - accuracy: 0.6640 - loss: 2.5105 - val_accuracy: 0.6639 - val_loss: 2.5087
Epoch 3/3
 5995/88000 ━━━━━━━━━━━━━━━━━━━━ 42:18 31ms/step - accuracy: 0.6639 - loss: 2.4663

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



88000/88000 ━━━━━━━━━━━━━━━━━━━━ 3138s 36ms/step - accuracy: 0.6649 - loss: 2.4654 - val_accuracy: 0.6626 - val_loss: 2.5000


In [4]:
lstm_model.save('/kaggle/working/lstm_model.h5')
bilstm_model.save('/kaggle/working/bilstm_model.h5')
gru_model.save('/kaggle/working/gru_model.h5')


In [6]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b181508129983acc0fca3b4db43ebd7327dad13434d53fbf567b313127e58eca
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from rouge_score import rouge_scorer
import nltk


# Load pre-trained models from .h5 files
lstm_model = tf.keras.models.load_model('/kaggle/working/lstm_model.h5')
bilstm_model = tf.keras.models.load_model('/kaggle/working/bilstm_model.h5')
gru_model = tf.keras.models.load_model('/kaggle/working/gru_model.h5')


In [16]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [20]:
import os
import numpy as np
from keras.models import load_model

# Paths to your model files
model_paths = {
    "LSTM": "/kaggle/working/lstm_model.h5",
    "BiLSTM": "/kaggle/working/bilstm_model.h5",
    "GRU": "/kaggle/working/gru_model.h5"
}

# Dummy input example (adjust based on your model's input shape)
dummy_input = np.random.rand(1, 512)  # Example shape (batch_size, max_length)

# Check, load, and test all models
for model_name, model_path in model_paths.items():
    print(f"\nChecking model: {model_name}")
    
    # Step 1: Check if the .h5 file exists
    if os.path.exists(model_path):
        print("Model file found.")
        
        # Step 2: Load the model
        try:
            model = load_model(model_path)
            print("Model loaded successfully.")
            
            # Step 3: Check the model summary
            model.summary()
            
            # Step 4: Pass dummy input to the model
            try:
                dummy_output = model.predict(dummy_input)
                print("Model prediction successful.")
                print("Output shape:", dummy_output.shape)
            except Exception as e:
                print(f"Error during prediction: {e}")

        except Exception as e:
            print(f"Error loading model: {e}")
    else:
        print("Model file not found.")



Checking model: LSTM
Model file found.
Model loaded successfully.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (1, 512, 128)          │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (1, 512, 256)          │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (1, 512, 20000)        │     5,140,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,094,242 (30.88 MB)

 Trainable params: 8,094,240 (30.88 MB)

...

 Total params: 7,996,450 (30.50 MB)

 Trainable params: 7,996,448 (30.50 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Model prediction successful.
Output shape: (1, 512, 20000)
